In [2]:
# Importing the dataframe
import pandas as pd
df = pd.read_csv('/kaggle/input/final-dataset/michelin_restaurants.csv', sep=',')
df

,index,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,0,O Me O Il Mare,Via Roma 45/47,Gragnano,80054,Italy,€€€€,"Italian Contemporary, Modern Cuisine","Known around the world as the town of pasta, G...","['Air conditioning', 'Interesting wine list', ...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 081 620 0550,http://omeoilmare.com
1,400,Da Politano,via Santuario 125,Boves,12012,Italy,€€,"Country cooking, Contemporary","At the kitchen in this restaurant, father-and-...","['Air conditioning', 'Terrace']","['Mastercard', 'Visa']",+39 0171 380383,http://www.hotelpolitano.it
2,100,Hostaria Viola,via Verdi 32,Castiglione delle Stiviere,46043,Italy,€€,"Mantuan, Lombardian",Having run this simple yet welcoming restauran...,"['Air conditioning', 'Car park', 'Wheelchair a...","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0376 670000,https://www.hostariaviola.com/
3,500,Saporium Firenze,lungarno Benvenuto Cellini 63r,Florence,50125,Italy,€€€€,Creative,A delightful small restaurant situated in a bu...,"['Air conditioning', 'Interesting wine list', ...","['Amex', 'Mastercard', 'Visa']",+39 055 212933,https://www.saporium.com/it/firenze/
4,600,Relais Blu,via Roncato 60,Massa Lubrense,80061,Italy,€€€,Mediterranean Cuisine,"Situated across the water from Capri, this res...","['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Mastercard', 'Visa']",+39 081 878 9552,https://www.relaisblu.com/it/restaurant/fine-d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,1598,Terramira,piazza della Vittoria 13,Capolona,52010,Italy,€€€€,"Contemporary, Tuscan","Having gained valuable experience elsewhere, t...","['Air conditioning', 'Great view']","['Amex', 'Mastercard', 'Visa']",+39 0575 420989,https://terramira.it
1977,1497,Serpillo,via di Mezzo 1,Bevagna,6031,Italy,€€,Italian Contemporary,Situated in a charming medieval village which ...,['Terrace'],"['Mastercard', 'Visa']",+39 366 711 8212,https://serpillo.it
1978,1599,L'Ostreria Fratelli Pavesi,Località Gariga 8,Podenzano,29027,Italy,€€,"Country cooking, Modern Cuisine",Overlooking a 20C courtyard typical of the Bas...,"['Air conditioning', 'Car park', 'Terrace']","['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 0523 524077,https://www.ostreria.it/
1979,1498,Il Ristorantino - Da Dino,via XXV Aprile 11,Anzola dell'Emilia,40011,Italy,€€,"Emilian, Traditional Cuisine",Tucked away in an unexpected location in a res...,['Air conditioning'],"['Amex', 'Dinersclub', 'Mastercard', 'Visa']",+39 051 732364,https://www.ristorantinodadino.it/


In [3]:
# Converting the postalCode values into integers in order to use them to detect the regions and 
# the coordinates 
import pandas as pd
import numpy as np

df['postalCode'] = df['postalCode'].apply(lambda x: int(x) if pd.notnull(x) else np.nan)

# Convert column to integer type, handling NaNs

df['postalCode'] = df['postalCode'].astype('Int64') 

### ****REQUEST 1/2****
> **Geocode Locations: Collect information on unique restaurant locations in Italy (in the format of City and Region). You can use tools such as Google API, OpenStreetMap, or a pre-defined list to retrieve representative coordinates for each region.**

To collect information regarding the cities, regions, and geographic coordinates, we employed a Google API tool, implemented in Python using the `Geocode` and `reverse geocode` libraries. 
This approach allowed us to efficiently retrieve and process geographic data, enabling us to convert addresses into latitude and longitude and vice versa.
To maximize the accuracy of the location data we designed our inputs to include the `address`,`city`,`postal code` and `country`, ensuring precise detection of coordinates.
Additionally, we conducted multiple tests using different combinations of inputs, firstly by implementing only the `city`,`postal code` and `country` and later using just `postal code`and `country`.\
Instead to retrieve information regarding the region we just implemented the `coordinates`. 
Despite these efforts, we encountered some errors related to the spelling of certain region names. To address these inaccuracies, we implemented systematic correction procedures, ensuring that all data was thoroughly verified and rectified.

In [4]:
# installing and importing the necessary packages. 
!pip3 install reverse-geocode
from geopy.geocoders import Nominatim
import reverse_geocode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.6 MB/s eta 0:00:0000:0100:01


In [5]:
# Correcting the name of the cities in order to be more comprehensible for the geocode package
df['city'] = df['city'].str.replace('.',' ')
df['city'] = df['city'].str.replace('_',' ')
df['city']

0                          Gragnano
1                             Boves
2        Castiglione delle Stiviere
3                          Florence
4                    Massa Lubrense
                   ...             
1976                       Capolona
1977                        Bevagna
1978                      Podenzano
1979             Anzola dell'Emilia
1980                       Cartosio
Name: city, Length: 1981, dtype: object

In [8]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

# Defining a function capable of detecting the coordinates foe each restaurant
def get_coordinates(row):
    geolocator = Nominatim(user_agent='restaurant', timeout=10) 
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    
    # full address
    full_address = f"{row['address']}, {row['city']}, {row['postalCode']}, Italy"
    location = geocode(full_address)
    if location:
        return location.latitude, location.longitude

    # city and postalCode only
    partial_address = f"{row['city']}, {row['postalCode']}, Italy"
    location = geocode(partial_address)
    if location:
        return location.latitude, location.longitude

    # only postalcode
    minimal_address = f"{row['postalCode']}, Italy"
    location = geocode(minimal_address)
    if location:
        return location.latitude, location.longitude

    return (None, None)

# Apply the function to the dataframe
df['coordinates'] = df.apply(get_coordinates, axis=1)

# Count remaining missing values to verify how many present null values
missing_count = df['coordinates'].isnull().sum()
print(f"Remaining unlocated coordinates: {missing_count}")


KeyboardInterrupt: 

The function works correctly, providing all the information needed. 
No missing values are present

In [ ]:
# separating the coordinates values in latitude and longitude and storing these new columns in the dataframe
df['latitude'] = df['coordinates'].apply(lambda x: x[0] if x else None)
df['longitude'] = df['coordinates'].apply(lambda x: x[1] if x else None)

In [ ]:
# Displaying rows with missing coordinates 
missing_coordinates = df[(df['latitude'].isnull()) | (df['longitude'].isnull())]
print("Rows with missing coordinates:")
print(missing_coordinates)

In [ ]:
# Filter rows where 'coordinates' is None
df_none_coordinates = df[df['coordinates'].isnull()]

In [ ]:
# Defining a function that can detect the region of the restaurants based on the coordinates
def region(coordinates):
    if isinstance(coordinates, (tuple, list)) and len(coordinates) == 2:
        try:
            state = reverse_geocode.get(coordinates)['state']
            return state
        except Exception as e:
            return None
    else:
        return None

# Defining the region column for the dataframe. 
df['Region'] = df['coordinates'].apply(region)

In [ ]:
# correcting the names of the regions and removing the wrong regions 
correction = {'Piedmont':'Piemonte','Basilicate':'Basilicata','Lombardy':'Lombardia','Tuscany':'Toscana','Sicily':'Sicilia','Apulia':'Puglia',
             'The Marches':'Marche', 'Sardinia':'Sardegna','Aosta Valley':"Valle D'Aosta", 'Basilicate':'Basilicata', "Provence-Alpes-Côte d'Azur":'Liguria'}

df['Region'] = df['Region'].replace(correction)
df = df[df['Region'] != "South Carolina"]

In [ ]:
# Displaying the new dataframe  
df

The package used for retrieving geolocation information, such as the coordinates and the regions, is `geopy`, a powerful Python library designed for geocoding, indeed it converts an address to coordinates. The package is also designed for reverse geocoding, by converting coordinates to an address. 

* `geopy`, interacts with multiple geolocation services, such as Nominatim (OpenStreetMap), Google Maps, Bing, and others. 
* `geopy` has allowed us to handle large-scale geolocation needs, enhancing our data's accuracy and usability by enriching it with geographic information.



### ****REQUEST 3****
> **Map Setup: Use a mapping library like plotly or folium to create a visual display of restaurants by region.**

#### ****CREATING MAP OF ITALY****

In [ ]:
from folium.plugins import MarkerCluster
import folium
# Defining the coordinates of italy 

italy_longitude = 12.56738
italy_latitude = 41.87194

# Plotting the map 
mappa = folium.Map(location = [italy_latitude, italy_longitude], zoom_start = 5, tiles='OpenStreetMap')
marker_cluster = MarkerCluster().add_to(mappa)

# adding marker
for idx, row in df.iterrows():
    folium.Marker(
        location=(row['latitude'], row['longitude']),
        popup=f"<b>{row['restaurantName']}</b><br>{row['address']}, {row['city']}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(marker_cluster)
    
mappa


#### Displaying the regions that present the several restaurants

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

region_counts = df['Region'].value_counts()

# Setting the figure, the style and the color palette
sns.set(style="whitegrid")
palette = sns.color_palette("icefire", len(region_counts))
plt.figure(figsize=(14, 10))

# Horizontal bar plot with Seaborn for smoother style
sns.barplot(
    x=region_counts.values,
    y=region_counts.index,
    palette=palette
)

# Adding titles and labels
plt.title('Number of Restaurants per Region in Italy', fontsize=18, fontweight='bold', color='#4a4a4a')
plt.xlabel('Number of Restaurants', fontsize=14, color='#4a4a4a')
plt.ylabel('Regions', fontsize=14, color='#4a4a4a')

# Customization of the grid
plt.xticks(fontsize=12, color='#4a4a4a')
plt.yticks(fontsize=12, color='#4a4a4a')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Removing the top and right spines for a cleaner look
sns.despine(left=True, bottom=True)

# Showing the plot
plt.show()


The region with the highest amount of restaurant is Lombardia. From the plot the regions with most restaurants are located mainly in the north italy. 

****Plotting a plot based on the region and the price ranges that each restaurant in the region has****

In [ ]:
# Defining a new variable with the region and the counts for each of the region. 
region_counts_df = region_counts.reset_index()
region_counts_df.columns = ['Region', 'Counts'] 
print(region_counts_df)
# looking at the price range of each restaurant
df['priceRange']
# Transforming the symbol into number, $ = 1, $$ = 2, $$$=3, $$$$=4 which represent the price range for each of the restaurants
df['price'] = df['priceRange'].apply(lambda x: len(x) if x else None)
region_counts_df['price'] = df['price']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Grouping by Region and Price to count the number of restaurants
region_counts = df.groupby(['Region', 'price']).size().unstack(fill_value=0)

# Using a seaborn color palette for more appealing colors
colors = sns.color_palette("Set2", n_colors=4) 

# Plotting stacked bar chart
fig, ax = plt.subplots(figsize=(10, 6))

price_levels = [1,2, 3, 4, 5] 
for i, price_level in enumerate(price_levels):
    if price_level in region_counts.columns:
        # Bottom layer is cumulative sum of previous layers
        bottom = region_counts[price_levels[:i]].sum(axis=1) if i > 0 else 0
        ax.bar(region_counts.index, region_counts[price_level], bottom=bottom, color=colors[i], label=f"{price_level}-Level of price")
        
# Add labels and title
ax.set_xlabel("Regions")
ax.set_ylabel("Number of Restaurants")
ax.set_title("Cheapest to Priciest Restaurants by Region")
ax.legend(title="Price Rating")

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Display the plot
plt.tight_layout()
plt.show()

The plot is showing the amount of restaurant in each region based on the price ranges, each color corresponds to a specific range. 
Lombardia is the region with the most restaurant and most of them are of price range = 2/3 meaning a bit pricey but not much. 
Most of them are in yellow, showing a medium range

In [ ]:
# Defining color based on price range --> plot shows little dots, each represents a restaurant
# with different colors. 
price_color_map = {
    '€': 'green',
    '€€': 'yellow',
    '€€€': 'orange',
    '€€€€': 'red'
}

# map centered around Italy
m = folium.Map(location=[italy_latitude, italy_longitude], zoom_start=6)

# Add markers to the map
for _, row in df.iterrows():
    color = price_color_map.get(row['priceRange'], 'gray')  # Default to gray if price is not in the map
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=8,  # Marker size
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"<b>{row['restaurantName']}</b><br>Price: {row['price']}<br>Address: {row['address']}, {row['city']}",
    ).add_to(m)

# Add a legend for price ranges
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 140px; 
                 border:2px solid grey; background-color:white; z-index:9999; font-size:14px;
                 font-family:Arial, Helvetica, sans-serif; padding: 10px;">
        <b>Price Range</b><br>
        <i style="background-color:green; width:20px; height:20px; display:inline-block; margin-right:5px;"></i> €<br>
        <i style="background-color:yellow; width:20px; height:20px; display:inline-block; margin-right:5px;"></i> €€<br>
        <i style="background-color:orange; width:20px; height:20px; display:inline-block; margin-right:5px;"></i> €€€<br>
        <i style="background-color:red; width:20px; height:20px; display:inline-block; margin-right:5px;"></i> €€€€<br>
     </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))
m.save("restaurant_map.html")
m

#### ****Plot Top-K Restaurants: Use the custom score from Step 3 to select the top-k restaurants for display.****


In [ ]:
import folium
import pandas as pd

# Define the top-K parameter
K = 5  # Select the top 5 restaurants
top_restaurants = df.nlargest(K, 'Score')  # Filter top-K restaurants by Score

# Initialize a folium map centered on Italy
map_center = [italy_latitude, italy_longitude]  # Rome's coordinates as a central point
m = folium.Map(location=map_center, zoom_start=6)

# Function to choose color based on price range
def get_color(price_range):
    colors = {"€": "green", "€€": "blue", "€€€": "orange", "€€€€": "red"}
    return colors.get(price_range, "gray")

# Add all restaurants to the map
for _, row in df.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"{row['city']} ({row['priceRange']}): Score {row['Score']}",
        icon=folium.Icon(color=get_color(row["priceRange"]))
    ).add_to(m)

# Highlight top-K restaurants with larger markers and a unique icon
for _, row in top_restaurants.iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=f"Top-{K} {row['city']} ({row['priceRange']}): Score {row['Score']}",
        icon=folium.Icon(color="purple", icon="star")
    ).add_to(m)

# Display the map
m.save("italy_restaurants_map.html")
m


# ALGORITHM QUESTION


#### ***1.Write the pseudocode for an algorithm that solves this problem.***

> Input n #number of packages
Initializing packages as an empty list to be filled later with the coordinates of each package

##### Getting the coordinates of the packages and appending them in the list:
> For coordinates from 1 to n:
    Input x, y  # Coordinates of the package
    Append (x, y) to packages

##### Sorting the packages by x first, then by y:
> Sort packages by x and y

##### Assigning variables: 
> Assigning the starting position as (0, 0), where we shouldnt have any gift
Initializing path as an empty string to then being filled with the letters R (right) and U (up)
Initializing result variable as "YES", this variable is meant to assume that we collected all the needed packages, so in the cases  

##### Checking and collecting each package with a for loop:
> For each (xi, yi) in packages:
    If xi < start.x OR yi < start.y:
        result = "NO"  # If moving backward we should stop and the result changes. 
        Breaking the loop to stop the cheking
    instead in the case in which we can actually move to right and up we keep going with the loop
    `If xi > start.x:
        Add 'R' (xi - start.x) # R is multiplied for the difference between the xi and the starting x, this shows how many steps to the right we are doing.
    
    If yi > start.y:
        Add 'U' (yi - start.y)  # U is multiplied for the difference between the yi and the starting y, this shows how many steps to the right we are doing.
    
    Update start to (xi, yi) ` # Updating the position so to continue with the loop

##### Checking the Result:
> If result is "NO":
    Print "NO"
Else:
    Print "YES"
    Print path # Printing the entire path 


#### ***2. Proving the algorithm is correct***

To prove that the algorithm is correct, we make some adjustments and give as input the following information: \
Input: \
3 \
5 \
1 3 \
1 2 \
3 3 \
5 5 \
4 3 \
2 \
1 0 \
0 1 \
1 \
4 3 


Which would give as an output: \
YES \
RUUURRRRUU \
NO \
YES \
RRRRUUU 

In [ ]:
n = int(input()) # number of test cases
final = []
for _ in range(n): 
    packages = [] # we put the packages list in the for loop, so to reset the list every loop 
    m = int(input())
    for _ in range(m): # number of packages
        x, y = map(int, input().split()) # coordinates of the packages
        packages.append((x,y)) # list of all the coordinates of the packages list[[(0,1),(0,9)]]
    
    # ordering the packages 
    packages.sort()

    start = (0,0)
    path = ''
    result = 'YES' # initially is yes
    for xi,yi in packages: 
        if xi < int(start[0]) or yi < int(start[1]): # if one package cannot be found we need to stop the calculation
            result = 'NO' 
            break # we stop the checking because we cannot reach all the packages 
        if xi > start[0]: # if xi greater than the x of the first package
            path += 'R' * (xi - start[0]) # since xi is greater than the starting point x,0, we more to the right and we add R to path
            # but we multiply it for how many times we move to the right 
        if yi > start[1]: 
            path += 'U' * (yi - start[1]) # same logic as before
            
        start = (xi,yi) # updating the starting point so to move on 
        
    if result == 'YES': # to check if we reached stopped or not 
        final.append("YES")
        final.append(path)
    else: 
        final.append("NO")

for x in final:
    print(x)

#### ***3. Compute the time complexity of your algorithm in Big O notation. Break down the steps involved in the algorithm, and explain which parts contribute most to the overall time complexity.***

The big O Notation is a way to describe how the time and space needed by an algorithm grows as the size of the input increases.
The total notation of the function is \(O(`nlogn`) \)

## **ANALYSIS**
1. **Input**: At the beginning we give as input the values `n` and `m`, which have each a complexity time of `n`: \( O(`1`) \), `m`: \( O(`1`) \)
2. **First for loops**: The first loop iterates `n` times, so it actually present a time complexity of `n`: \( O(`n`) \). 
3. **Second for loop**: The second loop iterates `m` times, so it present a time complexity of `m`: \( O(`n`) \).
4. **Sorting**: We then follow with the sorting of the list that presents all the coordinates. In the best case scenarios, the time complexity is \(O(`n`) \), but in the worst scenarios the complexity is \(O(`n*mlogn * m`) \). We add the m because we also need to consider the second loop that iterates m times.

For the second part, 
1. **Assigning**: we have the assigning of several variables which have a complexity of  `n`: \( O(`1`) \) because it is an operation that take the same amount of time all the times. 
2. **For loop**: The following for loop checks each package after sorting, determining a complexity of \( O(`m`) \).
3. **If Statements**: The 4 if statements perform a constant-time check regardless of the size of the input, indeed the if statements evaluates a condition thus it present a complexity of `n`: \( O(`1`) \) . 
4. **Final for loop**: the last for loop iterates over the list of paths, collected throughout the execution of the code. Considering that n represent the number of test cases. The total time complexity is  \( O(`n`) \)
### **CONCLUSION**
To calculate the total time complexity of the entire code, we need to combine the complexities of each part while considering the dominant terms.
* For the first part: In the worst case, the dominant term is \( O(`n*mlogn*m`) \) 
* For the second part: the variable and if assignments are not considered since they present a complexity of n(1), so the overall complexity of the second part is \( O(`m`) \) . 

#### Thus the total complexity of the code is \( O(`n*mlogn*m`) \) .

#### ***4.Ask an LLM tool (such as ChatGPT, Claude AI, Gemini, Perplexity, etc.) to evaluate the time complexity of your code using Big O notation. Is the assessment accurate? If it differs from your previous analysis, which would be correct? Please explain your reasoning.***

For the AI the code's complexity is the following:

1. **Reading the Input**:
   - Reading the number of test cases `n`: \( O(1) \)
   - Reading the number of packages `m` for each test case: \( O(n) \)
   - Reading the coordinates `x, y` for each package: \( O(n \times m) \)

2. **Sorting the Packages**:
   - Sorting the `packages` list: \( O(n \times m \log (n \times m)) \)

3. **Calculating the Path**:
   - Iterating through the sorted `packages` list: \( O(n \times m) \)
   - Performing constant-time operations (comparisons, string concatenation) within the loop: \( O(n \times m) \)

4. **Printing the Result**:
   - Iterating through the `final` list: \( O(n) \)
   - Printing each element: \( O(n) \)

Combining the complexities, the overall time complexity of the code is:

$$ O(1) + O(n) + O(n \times m) + O(n \times m \log (n \times m)) + O(n \times m) + O(n) + O(n) = O(n \times m \log (n \times m)) $$

The dominant term in the time complexity is \( O(n \times m \log (n \times m)) \), which is the complexity of sorting the `packages` list.

### ****Our time complexity consideration is the same as the AI's****

#### ***5.Assume now that the robot can also move towards the left or downwards, and consider the greedy approach: from the current location go to the closest package. Notice that now we can always collect all packages. Prove that the greedy algorithm is optimal (i.e., it minimizes the total distance traveled), or provide a counterexample showing that it is not.****


The greedy approach is an approach for solving problems by selecting the best option available. To prove that, we present an example.

---

### Proving That the Greedy Approach Always Yields the Optimal Solution

**Example of a scenario where the greedy algorithm is not suitable**:  
Consider a robot starting at the origin (0, 0) with three packages located at the following coordinates:

- Package A: (1, 3)  
- Package B: (4, 1)  
- Package C: (2, 2)

#### Assumptions:
The robot can move in any direction (up, down, right, and left).

---

### Greedy Algorithm Steps:

1. **Initial Position**: (0, 0)

2. **Distances to Packages**:
   - **Distance to Package A**:  
     \begin{equation} \tag{1}
     d_A = \sqrt{(1-0)^2 + (3-0)^2} = \sqrt{1 + 9} = \sqrt{10} \approx 3.16
     \end{equation}
   - **Distance to Package B**:  
     \begin{equation} \tag{2}
     d_B = \sqrt{(4-0)^2 + (1-0)^2} = \sqrt{16 + 1} = \sqrt{17} \approx 4.12
     \end{equation}
   - **Distance to Package C**:  
     \begin{equation} \tag{3}
     d_C = \sqrt{(2-0)^2 + (2-0)^2} = \sqrt{4 + 4} = \sqrt{8} \approx 2.83
     \end{equation}

   **Step 1 Decision**: The robot selects Package C as it is the closest.

3. **Next Steps**:  
   Recalculate distances from the robot's new position (2, 2).  
   - **Distance to Package A**:  
     \begin{equation} \tag{4}
     d_{C \to A} = \sqrt{(1-2)^2 + (3-2)^2} = \sqrt{1 + 1} = \sqrt{2} \approx 1.41
     \end{equation}
   - **Distance to Package B**:  
     \begin{equation} \tag{5}
     d_{C \to B} = \sqrt{(4-2)^2 + (1-2)^2} = \sqrt{4 + 1} = \sqrt{5} \approx 2.24
     \end{equation}

   **Step 2 Decision**: The robot selects Package A as it is now the closest.

---

### **Total Distance with Greedy Approach**:
- Move to Package C:  
  \begin{equation} \tag{6}
  d_C = \sqrt{8}
  \end{equation}
- Move to Package A:  
  \begin{equation} \tag{7}
  d_{C \to A} = \sqrt{2}
  \end{equation}

**Total Distance (Greedy)**:  
\begin{equation} \tag{8}
d_C + d_{C \to A} = \sqrt{8} + \sqrt{2} \approx 2.83 + 1.41 \approx 4.24
\end{equation}

---

### The Optimal Path:

Instead, if the robot chooses Package A as the first package:

1. **Move to Package A (1, 3)**:  
   \begin{equation} \tag{9}
   d_A = \sqrt{10} \approx 3.16
   \end{equation}

2. **Move from Package A (1, 3) to Package B (4, 1)**:  
   \begin{equation} \tag{10}
   d_{A \to B} = \sqrt{(4-1)^2 + (1-3)^2} = \sqrt{9 + 4} = \sqrt{13} \approx 3.61
   \end{equation}

---

### **Total Distance with Optimal Path**:
- Move to Package A:  
  \begin{equation} \tag{11}
  d_A = \sqrt{10}
  \end{equation}
- Move to Package B:  
  \begin{equation} \tag{12}
  d_{A \to B} = \sqrt{13}
  \end{equation}

**Total Distance (Optimal)**:  
\begin{equation} \tag{13}
\sqrt{10} + \sqrt{13} \approx 3.16 + 3.61 \approx 6.77
\end{equation}

---

### Conclusion:
Thus, the greedy algorithm does yield the optimal path, as it leads to a total distance of approximately **4.24**, while the "optimal path" yields a total distance of approximately **6.77**.

#### The greedy algorithm has proven to yield the optimal path in some cases, but not always.
